In [ ]:
import pandas as pd
import numpy as np
import random
from matplotlib import pyplot
import seaborn as sns


#Изучение данных

In [ ]:
data = pd.read_csv("titanic.csv")
data.head()

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
data.dropna().shape

In [ ]:
df = data.drop("PassengerId",axis=1)
df["Survived"].corr(df["Pclass"])

In [ ]:
df.isnull().sum()
sns.heatmap(df.isnull(), cbar = False).set_title("Missing values heatmap") # визуализация пропущенных значений

In [ ]:
df.nunique().plot(kind = "bar")


In [ ]:
listA = []
listD = []
for i in range(1,4):
    listA.append(df[(df["Survived"] == 1) & (df["Pclass"] == i)].shape[0])
    listD.append(df[(df["Survived"] == 0) & (df["Pclass"] == i)].shape[0])



In [ ]:
names = ['class1','class2','class3']

pyplot.bar(names,listA,label = 'alive',width=0.5)
pyplot.bar(names,listD,alpha = 0.2,label = "unknown",width=0.5,color = 'red')
pyplot.legend()

Видно, что в 3ем класе погибших процентно больше, чем в первых двух.

In [ ]:
med_age = np.mean(data["Age"])
med_age = float(round(med_age))
data["Age"] = data["Age"].fillna(med_age)

In [ ]:
fig_sex = (data.Sex.value_counts(normalize=True)*100).plot.bar()


65% людей на корабле были мужчины. 35 - женщины.

In [ ]:
male_pr = round((data[data.Sex == 'male'].Survived == 1).value_counts()[1]/len(data.Sex) * 100, 2)
female_pr = round((data[data.Sex == 'female'].Survived == 1).value_counts()[1]/len(data.Sex)*100,2)
sex_survived = pd.DataFrame({"Percentage Survived":{"male":male_pr,"female":female_pr},"Percentage not survived":{"male":100-male_pr, "female":100-female_pr}})
sex_survived.plot.barh()

Около 85 процентов мужчин погибли по сравнению с 70 процентами женщин. Рассмотрим признак fare, возможно он выявит еще лучшую зависимость между затратами и возможным выживанием.

In [ ]:
data.Fare.describe()

fareDf = pd.cut(data["Fare"],bins=[0,7.90,14.45,31.28,120], labels=['Low','Mid','High_Mid','High'])
T =sns.countplot(x= fareDf,hue = data.Survived).legend(labels = ["Unknown","Alive"])

Действительно, есть сильная зависимость между уровнем благосостояния и шансом на спасение.

In [ ]:
Embarked = sns.countplot(x = data.Embarked,hue=data.Survived)
Embarked.set_xticklabels(["Southampton","Cherbourg","Queenstown"])
Embarked.legend(labels = ["Unknown","Alive"])
Embarked.grid()

Видно,что больше 70% людей совершили посадку в Southampton. Людей погрузившихся в Cherbourg выжило больше, чем умерло.

## Логистическая регрессия:
Для начала, отделим категориальные признаки от вещественных.

In [ ]:
X = data.drop(["Survived","PassengerId","Name","Ticket","Cabin"],axis=1) # лишние признаки, отделение целевой переменной
y = data['Survived']

In [ ]:
X_numeric_cols = ["Pclass","Age","SibSp","Parch","Fare"] # числовые
X_categorial_cols = list(set(X.columns.values.tolist())- set(X_numeric_cols))# категориальные
X_cat = X[X_categorial_cols].fillna('NA').astype('str')
X_nums = X[X_numeric_cols]

In [ ]:
X.info() # в колонке Embarked 2 пропущенных значения,заменим на Southampton, поскольку там село наибольшее кол-во людей
X.Embarked = X.Embarked.fillna('S')

In [ ]:
from sklearn.linear_model import LogisticRegression as LR
from sklearn.feature_extraction import DictVectorizer as DV
encoder = DV(sparse=False)
X_cat_oh = encoder.fit_transform(X_cat.T.to_dict().values())


##### Закодировали категориальные признаки методом one-hot encoding, теперь разобьем нашу выборку на обучающую и тестовую.
Обучим Логистическую регрессию.

In [ ]:
from sklearn.model_selection import train_test_split
(X_train,X_test,y_train,y_test) = train_test_split(X_nums,y,test_size=0.3,random_state=0)

(X_train_cat_oh,X_test_cat_oh) = train_test_split(X_cat_oh,test_size=0.3, random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
param_grid = {'C': [0.01, 0.05, 0.1, 0.5, 1, 5, 10,20,25,50,100,400]}
cv = 100

X_train_stacked = np.hstack((X_train,X_train_cat_oh))
estimator = LogisticRegression(max_iter=10000)
optimizer = GridSearchCV(estimator,param_grid,cv=cv)

In [ ]:
%%time
optimizer.fit(X_train_stacked,y_train)

In [ ]:
print(optimizer.best_estimator_)
print(optimizer.best_score_)
print(optimizer.best_params_)

In [ ]:
X_test_stacked = np.hstack((X_test,X_test_cat_oh))
predict = optimizer.predict_proba(X_test_stacked)


In [ ]:
roc_auc_test = roc_auc_score(y_test,predict[:,1])
roc_auc_test

Получили хороший результат согласно roc_auc_score, однако попробуем улучшить модель. Самое простое, что следует проверить - балансировка классов.


In [ ]:
print(np.sum(y_train == 0))
np.sum(y_train == 1)


Видно, что классы несбалансированны, попробует исправить это исправить
